# Helmet Detection

## Install

In [ ]:
# yolov5 github repo clone
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd

In [ ]:
# pip module
!pip install -U -r /content/content/yolov5/requirements.txt
!pip install wandb

# !pip install wandb

In [ ]:
import torch

#check pytorch version, cuda device properties 
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# dataset
"""
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="write your apikey")
project = rf.workspace("bike-helmets").project("bike-helmet-detection-2vdjo")
dataset = project.version(2).download("yolov5")
"""

## Train 


**arguments**:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive 
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training
![이미지](https://user-images.githubusercontent.com/26833433/155040763-93c22a27-347c-4e3c-847a-8094621d3f4e.png)

In [ ]:
from glob import glob

# image path list
train_img_list = glob('./helmet/dataset/train/images/*.jpg') 
valid_img_list = glob('./helmet/dataset/valid/images/*.jpg') 


# write on txt
with open('./train.txt', 'w') as f:
	f.write('\n'.join(train_img_list) + '\n')
    
with open('./valid.txt', 'w') as f:
	f.write('\n'.join(valid_img_list) + '\n')

In [ ]:
# train helmet data for 30 epochs
# without evolve
%%time
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 30 --data /content/dataset/data.yaml --cfg /content/yolov5/models/yolov5x.yaml --weights yolov5x.pt --name yolov5x_results --cache python train.py 


In [ ]:
# train helmet data for 30 epochs
# with evolve
%%time
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 30 --data /content/dataset/data.yaml --cfg /content/yolov5/models/yolov5x.yaml --weights yolov5x.pt --name yolov5x_results --cache python train.py --evolve 


## Val

In [ ]:
!python /content/yolov5/val.py --data /content/dataset/data.yaml --weights /content/yolov5/runs/train/yolov5x_results3/weights/best.pt

## Test

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/yolov5x_results3/weights/best.pt --img 416 --conf 0.4 --source /content/dataset/test/images --save-txt --save-conf 

### test custom

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/yolov5x_results3/weights/best.pt --img 416 --conf 0.25 --source /content/1.jpg --save-txt --save-conf

In [ ]:
# test mov
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/yolov5x_results3/weights/best.pt --img 416 --conf 0.25 --source /content/test.mov --save-txt --save-conf

In [ ]:
import pandas as pd
# get mAP
txt_path = "/content/content/yolov5/runs/detect/labels/1.txt"
def get_score(txt_path):
    txt_read = pd.read_csv(txt_path, sep=' ', header=None, index_col=0)
    score = txt_read.loc[0,5]
    return(score)

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

## save


In [ ]:
from google.colab import files
import os

dir_to_zip = '/content/yolov5/runs/train/yolov5x_results3/weights' #@param {type: "string"}
output_filename = 'weights_yolov5x.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )

In [ ]:
!zip -r /content/content/yolov5.zip /content/content/yolov5